In [60]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import spotipy

In [11]:
client_credentials_manager = SpotifyClientCredentials(client_id='82b857bc19b8402dbf050cc433158f42', client_secret='799bcde8e32b40ceaa19793845087b5f')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [12]:
results = sp.search(q = 'sezen-aksu', limit = 20)

In [25]:
results['tracks'].keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [40]:
album = []
track = []
date = []
for item in results['tracks']['items']:
    album.append(item['album']['name'])
    track.append(item['name'])
    date.append(item['album']['release_date'])
    
pd.DataFrame({
    'album':album,
    'track':track,
    'date':date
})

,album,track,date
0,Biraz Pop Biraz Sezen,Manifesto,2017-01-23
1,Biraz Pop Biraz Sezen,İhanetten Geri Kalan,2017-01-23
2,Gülümse,Vazgeçtim,1991-07-16
3,Düş Bahçeleri,Kaçın Kurası,1996-07-14
4,Öptüm,Vay,2011-05-23
5,Gülümse,Her Şeyi Yak,1991-07-16
6,Gülümse,Seni Kimler Aldı,1991-07-16
7,Deniz Yıldızı,Kutlama,2008-06-18
8,Biraz Pop Biraz Sezen,Köz,2017-01-23
9,Git,Git...,1986-09-02


In [48]:
results2 = sp.search('biraz-pop-biraz-sezen', limit = 20)

i = 1
for item in results2['tracks']['items']:
    print(i, item['name'])
    i += 1

In [52]:
results3 = sp.search('3 Büyükler', limit = 20)

i = 1
for item in results3['tracks']['items']:
    print(i, item['name'])
    i += 1

1 İmkansız
2 Ah Bu Şarkıların Gözü Kör Olsun
3 Gücüme Gidiyor Böyle Yaşamak - Version 1
4 Büyüleyen Gözlerinle
5 Ayaz Geceler
6 Rüyalarda Buluşuruz
7 Sevgi Dolu Şu Gönlüm
8 Göz Yaşımda Saklısın
9 Seni Ben Unutmak İstemedim Ki - Version 1
10 Biz Ayrılamayız
11 Kanımda Kıvılcım
12 Sevgi Bağları
13 Zulüm Taşıyan Gözler
14 Acı Hasret
15 Gücüme Gidiyor Böyle Yaşamak - Version 2
16 Güzelleri Kim Sevmez Ki
17 Bana Aşkı Öğreten Sensin
18 Seviyorum Sevmesende
19 Güzelsin
20 N'olur Hayatımdan Çıkma


In [57]:
lyrics_df = pd.read_csv('../assets/lyrics/all_lyrics_scraped_20180625-155423.csv', index_col = 0)

In [61]:
def remove_ads(text):
    pattern = r'(eval.*])'
    return re.sub(pattern, "", text)

def regularize_whitespace(text):
    whitepattern = r'\s+'
    return re.sub(whitepattern, " ", text)

def remove_and_reg(text):
    text = remove_ads(text)
    text = regularize_whitespace(text)
    return text

def check_for_english(text):
    text_words = text.split(" ")
    english_words = set(['I', "we're", "the", "an", "one", "to", "give", "love"])
    counter = 0
    for word in text_words:
        if word in english_words:
            counter += 1
    return counter
    

lyrics_df['text'] = lyrics_df['text'].map(remove_and_reg)

lyrics_df['english_score'] = lyrics_df['text'].map(check_for_english)

In [62]:
lyrics_df[lyrics_df['english_score'] == 0].shape

(41943, 5)

In [64]:
lyrics_df.shape[0] - lyrics_df[lyrics_df['english_score'] == 0].shape[0]

13742

In [67]:
tr_lyrics_df = lyrics_df[lyrics_df['english_score'] == 0]

In [69]:
len(tr_lyrics_df['artist'].unique())

1809

In [70]:
artists = tr_lyrics_df['artist'].unique()

In [75]:
artists[0]

'A P O'

In [77]:
import time
time.sleep(2)

In [84]:
tr_lyrics_df.reset_index(inplace = True)

In [104]:
query_a = tr_lyrics_df.loc[1000, 'artist']
query_b = tr_lyrics_df.loc[1000,'title']

In [105]:
query = query_a + " "+  query_b

In [116]:
results4 = sp.search(q = query, limit = 1)


print(results4['tracks']['items'][0]['name'])
print(results4['tracks']['items'][0]['album']['release_date'])

Bak Bir Erkek Ağlıyor
1987-02-04


In [120]:
results5 = sp.search(q = 'Ahmet burhan isik', limit = 1)

print(results5)

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=Ahmet+burhan+isik&type=track&offset=0&limit=1', 'items': [], 'limit': 1, 'next': None, 'offset': 0, 'previous': None, 'total': 0}}


In [115]:
pd.to_datetime(results4['tracks']['items'][0]['album']['release_date'])

Timestamp('1987-02-04 00:00:00')

In [109]:
tr_lyrics_df.loc[1000:1010, :]

,index,artist,title,album,text,english_score
1000,1498,Ahmet Selçuk İlkan,Bak Bir Erkek Ağlıyor,Seni Arıyorum,Son mektubun elimde Bana veda etmişsin Ayrılır...,0
1001,1499,Ahmet Selçuk İlkan,Bana Bunu Yapmayacaktın,Unutmaktan Geliyorum,şiir: bana bunu yapmayacaktın öyle sırtımdan v...,0
1002,1500,Ahmet Selçuk İlkan,Ben Aşkı Ölümsüz Bilenlerdenim (Ft. Ebru Yaşar),Söz,Ahmet Selçuk İlkan: Aradığım ne varsa bil ki s...,0
1003,1501,Ahmet Selçuk İlkan,Ben En Çok İnsandan Korktum (Ft. İntizar),Seni O Kadar Çok Sevdim Ki,Ahmet Selçuk İlkan: Durup durup bana sorma bun...,0
1004,1502,Ahmet Selçuk İlkan,Ben Seni Asla (Ft. Baha),Seni O Kadar Çok Sevdim Ki,Ahmet Selçuk İlkan: Sen hayatımın en kutsal aş...,0
1005,1503,Ahmet Selçuk İlkan,Bir Adam Gidiyor – Sitem (Ozan Kavasoğulları),Seni O Kadar Çok Sevdim Ki,Ozan Kavasoğulları: Kendi elinle düştün bu hal...,0
1006,1504,Ahmet Selçuk İlkan,Bir Televole Masalı,O Adam Benim,Hayat bir televole masalı değildir kızım! Sakı...,0
1007,1505,Ahmet Selçuk İlkan,Bu Şehir Ve Sen,Asi Bir Tutku,Ömrümün en güzel senelerini Alıpta gittiniz bu...,0
1008,1507,Ahmet Selçuk İlkan,Geçen Yıl Bu Zamanlar (Ft. İntizar),deli sevda,Ahmet Selçuk İlkan: Öyle bir vurdu ki hasretin...,0
1009,1508,Ahmet Selçuk İlkan,Gidiyorsun,O Adam Benim,"Ne yaptım ki sana böyle, Kaçar gibi gidiyorsun...",0


## OK it seems I got a plan here:
Psuedocode for getting a Spotify date column:
```
release_spotify = []
name_spotify = []
for each song in the Turkish-only lyrics:
    make a request to spotify
    ask for the top result
    if there is no top result:
        release_spotify.append(np.na)
        name_spotify.append(nap.na)
    else:
        release_spotify.append(request['tracks']['items'][0]['album']['release_date'])
        name_spotify.append(request['tracks']['items'][0]['name'])
        
```
This will produce two new columns: an album release date column (at least, Spotify's best guess) and the name of the top result (for verification purposes)

In [125]:
import numpy as np

In [128]:
release_spotify = []
name_spotify = []

songs = tr_lyrics_df['title']
artists = tr_lyrics_df['artist']
for song, artist in zip(songs, artists):
    r = sp.search(q = song+" "+artist, limit = 1)
    if len(r['tracks']['items']) == 0:
        release_spotify.append(np.nan)
        name_spotify.append(np.nan)
    else:
        release_spotify.append(r['tracks']['items'][0]['album']['release_date'])
        name_spotify.append(r['tracks']['items'][0]['name'])
    time.sleep(2)
    
output_df = pd.DataFrame({
    'release':release_spotify,
    'name': name_spotify,
    'real_name': songs
})

,release,name,real_name
500,NaN,NaN,Aldığım Her Nefesin
501,2013-04-22,Anma Adımı,Anma Adımı
502,NaN,NaN,Annem
503,NaN,NaN,Aşk Bilmecesi
504,NaN,NaN,Aşkının Sarhoşuyum
505,NaN,NaN,Bana Her Şey Seni Hatırlatıyor
506,1994-05-26,Bedel,Bedel
507,NaN,NaN,Beklenen Sensin
508,2013-04-22,Bekleyemedin mi,Bekleyemedin Mi
509,1989-06-14,Biliyorsun Bir Zamanlar,Biliyorsun Bir Zamanlar


# Discogs

In [1]:
import discogs_client

In [2]:
d = discogs_client.Client('TurkishSongNLP/0.1')

In [3]:
d.set_consumer_key('DtigwAKfRYZIbHOXXqHd', 'zssaHwVHWvlzOAmayWHJzWhuTaUYchpE')

In [7]:
d.get_authorize_url()

('iQASppqcVOEWWVJrrJbJOzVdBEiCtabppKMxLtqC',
 'MhiifeuJORwjGVzHcnOQSjOrXFPmhDcTBPUqFmPA',
 'https://www.discogs.com/oauth/authorize?oauth_token=iQASppqcVOEWWVJrrJbJOzVdBEiCtabppKMxLtqC')

In [8]:
d.get_access_token('YTorbcacsz')

('zkXBzNqGqAODjrBQhJYbJBfExrlSVNrLHAsbXkJN',
 'SDaxZXYZaYDHyQXuKvLFslnqnlLAUuLuWduUowUN')

In [20]:
results = d.search('kuzu kuzu', type='release')

In [36]:
results[0]

<Release 2348428 'Kuzu Kuzu'>

In [37]:
d.release(results[0].id).year

2001

In [39]:
d.release(d.search('kuzu kuzu', type='release')[0].id).year

2001

In [47]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import numpy as np

def remove_ads(text):
    pattern = r'(eval.*])'
    return re.sub(pattern, "", text)

def regularize_whitespace(text):
    whitepattern = r'\s+'
    return re.sub(whitepattern, " ", text).strip()

def remove_and_reg(text):
    text = remove_ads(text)
    text = regularize_whitespace(text)
    return text
def check_for_english(text):
    text_words = text.split(" ")
    english_words = set(['I', "we're", "the", "an", "one", "to", "give", "love"])
    counter = 0
    for word in text_words:
        if word in english_words:
            counter += 1
    return counter




lyrics_df = pd.read_csv('../assets/lyrics/all_lyrics_scraped_20180625-155423.csv', index_col = 0)

lyrics_df['text'] = lyrics_df['text'].map(remove_and_reg)
lyrics_df['english_score'] = lyrics_df['text'].map(check_for_english)
tr_lyrics_df = lyrics_df[lyrics_df['english_score'] == 0]

release_discogs = []

songs = tr_lyrics_df['title']

total_songs = len(songs)
i = 1

In [53]:
for song in songs:
    try:
        release_discogs.append(d.release(d.search(song, type='release')[0].id).year)
        print(d.release(d.search(song, type='release')[0].id).year)
    except:
        release_discogs.append(np.nan)
        print(d)
    time.sleep(1.2)
    
    

1979
2013
2013
1999


KeyboardInterrupt: 